In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/zillow-prize-1/zillow_data_dictionary.xlsx
/kaggle/input/zillow-prize-1/sample_submission.csv
/kaggle/input/zillow-prize-1/properties_2016.csv
/kaggle/input/zillow-prize-1/train_2017.csv
/kaggle/input/zillow-prize-1/properties_2017.csv
/kaggle/input/zillow-prize-1/train_2016_v2.csv


In [2]:
properties2017=pd.read_csv("/kaggle/input/zillow-prize-1/properties_2017.csv")


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
properties2016=pd.read_csv("/kaggle/input/zillow-prize-1/properties_2016.csv")
print(properties2016)


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


          parcelid  airconditioningtypeid  architecturalstyletypeid  \
0         10754147                    NaN                       NaN   
1         10759547                    NaN                       NaN   
2         10843547                    NaN                       NaN   
3         10859147                    NaN                       NaN   
4         10879947                    NaN                       NaN   
...            ...                    ...                       ...   
2985212  168176230                    NaN                       NaN   
2985213   14273630                    NaN                       NaN   
2985214  168040630                    NaN                       NaN   
2985215  168040830                    NaN                       NaN   
2985216  168040430                    NaN                       NaN   

         basementsqft  bathroomcnt  bedroomcnt  buildingclasstypeid  \
0                 NaN          0.0         0.0                  NaN   
1    

In [4]:
train2017=pd.read_csv("/kaggle/input/zillow-prize-1/train_2017.csv",parse_dates=['transactiondate'])


In [5]:

train2016=pd.read_csv("/kaggle/input/zillow-prize-1/train_2016_v2.csv",parse_dates=['transactiondate'])


In [6]:
submission=pd.read_csv("/kaggle/input/zillow-prize-1/sample_submission.csv")


In [7]:
def add_date_features(df):
    df["transaction_year"] = df["transactiondate"].dt.year
    df["transaction_month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month
    df["transaction_day"] = df["transactiondate"].dt.day
    df["transaction_quarter"] = (df["transactiondate"].dt.year - 2016)*4 +df["transactiondate"].dt.quarter
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

In [8]:
train2016 = add_date_features(train2016)
train2017 = add_date_features(train2017)

In [9]:
train2017

,parcelid,logerror,transaction_year,transaction_month,transaction_day,transaction_quarter
0,14297519,0.025595,2017,13,1,5
1,17052889,0.055619,2017,13,1,5
2,14186244,0.005383,2017,13,1,5
3,12177905,-0.103410,2017,13,1,5
4,10887214,0.006940,2017,13,1,5
...,...,...,...,...,...,...
77608,10833991,-0.002245,2017,21,20,7
77609,11000655,0.020615,2017,21,20,7
77610,17239384,0.013209,2017,21,21,7
77611,12773139,0.037129,2017,21,21,7


Merge Train with Properties

In [10]:
train2016 = pd.merge(train2016, properties2016, how = 'left', on = 'parcelid')
train2017 = pd.merge(train2017, properties2017, how = 'left', on = 'parcelid')

In [11]:
#Tax feature 2017

train2017.iloc[:, train2017.columns.str.startswith('tax')] = np.nan

In [12]:
print('Concat Train 2016 & 2017 ...')
train_df = pd.concat([train2016, train2017], axis = 0)
test_df = pd.merge(submission[['ParcelId']], properties2016.rename(columns = {'parcelid': 'ParcelId'}), how = 'left', on = 'ParcelId')


Concat Train 2016 & 2017 ...


In [13]:
import gc

del properties2016, properties2017, train2016, train2017
gc.collect();

In [14]:
train_df

,parcelid,logerror,transaction_year,transaction_month,transaction_day,transaction_quarter,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.027600,2016,1,1,1,1.0,NaN,NaN,2.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.168400,2016,1,1,1,NaN,NaN,NaN,3.5,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.004000,2016,1,1,1,1.0,NaN,NaN,3.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.021800,2016,1,2,1,1.0,NaN,NaN,2.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.005000,2016,1,2,1,NaN,NaN,NaN,2.5,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,10833991,-0.002245,2017,21,20,7,1.0,NaN,NaN,3.0,...,NaN,NaN,265000.0,NaN,2016.0,114000.0,NaN,NaN,NaN,6.037132e+13
77609,11000655,0.020615,2017,21,20,7,NaN,NaN,NaN,2.0,...,NaN,NaN,70917.0,NaN,2016.0,283704.0,NaN,NaN,NaN,6.037101e+13
77610,17239384,0.013209,2017,21,21,7,NaN,NaN,NaN,2.0,...,1.0,NaN,50683.0,NaN,2016.0,16522.0,NaN,NaN,NaN,6.111008e+13
77611,12773139,0.037129,2017,21,21,7,1.0,NaN,NaN,1.0,...,NaN,NaN,32797.0,NaN,2016.0,16749.0,NaN,NaN,NaN,6.037434e+13


In [15]:
#Remove missing value 
missing_perc_thresh = 0.98
exclude_missing = []
num_rows = train_df.shape[0]
for c in train_df.columns:
    num_missing = train_df[c].isnull().sum()
    if num_missing == 0:
        continue
    missing_frac = num_missing / float(num_rows)
    if missing_frac > missing_perc_thresh:
        exclude_missing.append(c)
print("We exclude: %s" % len(exclude_missing))


del num_rows, missing_perc_thresh
gc.collect();


We exclude: 15


In [16]:
#Remove unique values

exclude_unique = []
for c in train_df.columns:
    num_uniques = len(train_df[c].unique())
    if train_df[c].isnull().sum() != 0:
        num_uniques -= 1
    if num_uniques == 1:
        exclude_unique.append(c)
print("We exclude: %s" % len(exclude_unique))

We exclude: 9


In [17]:
exclude_other = ['parcelid', 'logerror','propertyzoningdesc']
train_features = []
for c in train_df.columns:
    if c not in exclude_missing \
       and c not in exclude_other and c not in exclude_unique:
        train_features.append(c)
print("We use these for training: %s" % len(train_features))

We use these for training: 42


In [18]:
print ("Define categorial features !!")
cat_feature_inds = []
cat_unique_thresh = 1000
for i, c in enumerate(train_features):
    num_uniques = len(train_df[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)
        
print("Cat features are: %s" % [train_features[ind] for ind in cat_feature_inds])

Define categorial features !!
Cat features are: ['transaction_year', 'transaction_month', 'transaction_day', 'transaction_quarter', 'airconditioningtypeid', 'buildingqualitytypeid', 'fips', 'heatingorsystemtypeid', 'propertycountylandusecode', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'yearbuilt', 'assessmentyear']


In [19]:
#Replace Nan to -999

print ("Replacing NaN values by -999 !!")
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

Replacing NaN values by -999 !!


In [20]:
print ("Training time !!")
X_train = train_df[train_features]
y_train = train_df.logerror
print(X_train.shape, y_train.shape)

Training time !!
(167888, 42) (167888,)


In [21]:
test_df['transactiondate'] = pd.Timestamp('2016-12-01') 
test_df = add_date_features(test_df)
X_test = test_df[train_features]
print(X_test.shape)

(2985217, 42)


In [22]:
#catboost Model

from catboost import CatBoostRegressor
from tqdm import tqdm

num_ensembles = 5
y_pred = 0.0
for i in tqdm(range(num_ensembles)):
    model = CatBoostRegressor(
        iterations=630, learning_rate=0.03,
        depth=6, l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=i)
    model.fit(
        X_train, y_train,
        cat_features=cat_feature_inds)
    y_pred += model.predict(X_test)
y_pred /= num_ensembles

  0%|          | 0/5 [00:00<?, ?it/s]


CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=4]=1.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [23]:
submission = pd.DataFrame({
    'ParcelId': test_df['ParcelId'],
})
test_dates = {
    '201610': pd.Timestamp('2016-09-30'),
    '201611': pd.Timestamp('2016-10-31'),
    '201612': pd.Timestamp('2016-11-30'),
    '201710': pd.Timestamp('2017-09-30'),
    '201711': pd.Timestamp('2017-10-31'),
    '201712': pd.Timestamp('2017-11-30')
}

In [24]:
for label, test_date in test_dates.items():
    print("Predicting for: %s ... " % (label))
    submission[label] = y_pred

Predicting for: 201610 ... 
Predicting for: 201611 ... 
Predicting for: 201612 ... 
Predicting for: 201710 ... 
Predicting for: 201711 ... 
Predicting for: 201712 ... 


In [25]:
submission.head(30)
submission.to_csv('Only_CatBoost.csv', float_format='%.6f',index=False)
